In [30]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

# Charger le fichier CSV
df = pd.read_csv("metadata.csv")  # Mets le bon chemin

# Afficher les premières lignes
print(df.head())


# Vérifier la distribution des classes
print(df['class'].value_counts())  # Supposons qu'il y ait une colonne "label" indiquant si c'est une tumeur ou non
print(df.columns)




   Unnamed: 0            image  class format mode          shape
0           0   Cancer (1).jpg  tumor   JPEG  RGB  (512, 512, 3)
1           1   Cancer (1).png  tumor    PNG    L     (300, 240)
2           2   Cancer (1).tif  tumor   TIFF  RGB  (256, 256, 3)
3           3  Cancer (10).jpg  tumor   JPEG  RGB  (512, 512, 3)
4           4  Cancer (10).tif  tumor   TIFF  RGB  (256, 256, 3)
class
tumor     2513
normal    2087
Name: count, dtype: int64
Index(['Unnamed: 0', 'image', 'class', 'format', 'mode', 'shape'], dtype='object')


In [ ]:

# la dataset ne contient pas les images donc je metd ce que jaurai fais .

image_folder = 


df["image_path"] = df["image"].apply(lambda x: os.path.join(image_folder, x))


df = df[df["image_path"].apply(os.path.exists)]




datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2, zoom_range=0.2)






KeyError: 'image'

In [28]:
# Séparer en ensembles d'entraînement et validation
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df["class"], random_state=42)

# Paramètres
img_size = (150, 150)
batch_size = 32

KeyError: 'class'

In [ ]:
# Chargement des images
train_gen = datagen.flow_from_dataframe(
    train_df,
    x_col="image_path",
    y_col="class",
    target_size=img_size,
    batch_size=batch_size,
    class_mode="binary",
    subset="training"
)

val_gen = datagen.flow_from_dataframe(
    val_df,
    x_col="image_path",
    y_col="class",
    target_size=img_size,
    batch_size=batch_size,
    class_mode="binary",
    subset="validation"
)


In [ ]:

# Visualisation de la distribution des classes
sns.countplot(x=train_df["class"])
plt.title("Distribution des classes")
plt.show()


In [ ]:

# Modèle CNN
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation="relu", input_shape=(150, 150, 3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation="relu"),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(1, activation="sigmoid")

In [ ]:
# Compilation
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])


In [ ]:
# Entraînement
history = model.fit(train_gen, validation_data=val_gen, epochs=10)


In [ ]:
# Courbes de performance
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()